# Melting and dissolving solids in buoyant fluid flows

This notebook demonstrates use of a second-order accurate phase field model for the melting and dissolution of solids in buoyant fluids.
The mathematics are developed in the paper by [Hester et al. (2020)](https://doi.org/10.1098/rspa.2020.0508), and the numerical implementation is done in version 3 of the [Dedalus code](https://doi.org/10.1098/rspa.2020.05080) adapted from an original code (courtesy of Eric Hester).

In [ ]:
import numpy as np
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)
import matplotlib.pyplot as plt
import time

In [ ]:
# Model parameters
Lx, Lz = 4, 2 # domain size
ν = 1e-2 # kinematic viscosity
κ = ν # thermal diffusivity
μ = ν # salt diffusivity
ϵ = 1e-2 # phase-field interface thickness
γ = 1e-2 # damping rate
L = 1 # Stefan number (latent heat)
β = 1.51044385 # Optimal damping proportionality, may underestimate at large ε
m = 0 # salinity induced melting temperature change
n = 1 # temperature-salinity buoyancy ratio
δ = 1e-2 # concentration forcing regularisation

In [ ]:
# Numerical parameters
Nx, Nz = 256, 256
dealias = 3/2
stop_sim_time = 1
timestepper = d3.RK222
max_timestep = 0.125
dtype = np.float64

In [ ]:
# Coordinates, Bases
coords = d3.CartesianCoordinates('x', 'z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=dealias)
zbasis = d3.ChebyshevT(coords['z'], size=Nz, bounds=(0, Lz), dealias=dealias)
x, z = dist.local_grids(xbasis, zbasis)
ex, ez = coords.unit_vector_fields(dist)


In [ ]:
# Fields
p = dist.Field(name='p', bases=(xbasis,zbasis))
T = dist.Field(name='T', bases=(xbasis,zbasis))
C = dist.Field(name='C', bases=(xbasis,zbasis))
f = dist.Field(name='f', bases=(xbasis,zbasis))
ft = dist.Field(name='ft', bases=(xbasis,zbasis))
u = dist.VectorField(coords, name='u', bases=(xbasis,zbasis))

tau_p = dist.Field(name='tau_p')
tau_T1 = dist.Field(name='tau_T1', bases=xbasis)
tau_T2 = dist.Field(name='tau_T2', bases=xbasis)
tau_C1 = dist.Field(name='tau_C1', bases=xbasis)
tau_C2 = dist.Field(name='tau_C2', bases=xbasis)
tau_f1 = dist.Field(name='tau_f1', bases=xbasis)
tau_f2 = dist.Field(name='tau_f2', bases=xbasis)
tau_u1 = dist.VectorField(coords, name='tau_u1', bases=xbasis)
tau_u2 = dist.VectorField(coords, name='tau_u2', bases=xbasis)

# Substitutions
lift_basis = zbasis.derivative_basis(1)
lift = lambda A: d3.Lift(A, lift_basis, -1)
dz = lambda A: d3.Differentiate(A,coords['z'])

tau_div_eq = ez@lift(tau_u1) + tau_p
tau_temp_eq = -κ*dz(lift(tau_T1)) + lift(tau_T2)
tau_conc_eq = -μ*dz(lift(tau_C1)) + lift(tau_C2)
tau_phas_eq = -γ*dz(lift(tau_f1)) + lift(tau_f2)
tau_mom_eq  = -ν*dz(lift(tau_u1)) + lift(tau_u2)

In [ ]:
# Volume penalty walls
mask = lambda x : 0.5*(1 + np.tanh(x/(2*ϵ)))
wall = dist.Field(name='wall', bases=(xbasis,zbasis))
wall['g'] = mask(-(x-Lx/20)) + mask(x-Lx*(1-1/20))

In [ ]:
# integral quantities
momentum = d3.integ(u)
heat = d3.integ(T) - L*d3.integ(f)
salt = d3.integ((1-f)*(1-wall)*C)

# boundary quantities
heat_flux = dz(T)(z=Lz) - dz(T)(z=0)

In [ ]:
# # simpler reformulation of the tau terms
# (d3.trace(ez*lift(tau_u1)) - ez@lift(tau_u1)).evaluate()['g'].max() # .6 faster
# (d3.div(ez*lift(tau_T1)) - dz(lift(tau_T1))).evaluate()['g'].max() # 2.6/7.3 ~ .35 faster

In [ ]:
# Problem
problem = d3.IVP([p, u, T, f, ft, C, 
                  tau_p, tau_T1, tau_T2, tau_f1, tau_f2, tau_C1, tau_C2, 
                  tau_u1, tau_u2, 
                  ], namespace=locals())

problem.add_equation("dt(f) - ft = 0")
problem.add_equation("div(u) + tau_div_eq = 0")
problem.add_equation("dt(T) - κ*div(grad(T)) - L*dt(f)              + tau_temp_eq = - (1-f)*u@grad(T) + T*u@grad(f)")
problem.add_equation("dt(C) - μ*div(grad(C))                        + tau_conc_eq = - u@grad(C) + (C*ft - μ*grad(f)@grad(C))/(1-f+δ) + (1-f)*μ*grad(wall)@grad(C)/(1-wall+δ)")
problem.add_equation("(5/6)*(L/κ)*dt(f) - (γ/ε)*div(grad(f))        + tau_phas_eq = -(1/ϵ**2)*f*(1-f)*((γ/ε)*(1-2*f) + (T+m*C))")
problem.add_equation("dt(u) - ν*div(grad(u)) + grad(p) - (T-n*C)*ez + tau_mom_eq  = - u@grad(u) - (ν/(β*ϵ)**2)*(f+wall)*u")

problem.add_equation("T(z=Lz) = 0")
problem.add_equation("dz(C)(z=Lz) = 0")
problem.add_equation("u(z=Lz) = 0")
problem.add_equation("f(z=Lz) = 1")
problem.add_equation("integ(p) = 0") # Pressure gauge
problem.add_equation("T(z=0) = 1")
problem.add_equation("dz(C)(z=0) = 0")
problem.add_equation("u(z=0) = 0")
problem.add_equation("f(z=0) = 0")

In [ ]:
# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time

In [ ]:
f.change_scales(1)
f['g'] = mask(z-Lz/2)
xx, zz = x+0*z, 0*x+z

u.change_scales(1)
u['g'] = 0
# u['g'] = (ex['g']*(1-f['g'])*z*(Lz/2-z))
# u['g'][0][zz>Lz/2] = 0

# Initial conditions
T.change_scales(1)
# T.fill_random('g', seed=42, distribution='normal', scale=2e-2) # Random noise
# T['g'] += 1 - 2*z/Lz
T['g'] = (1-f['g'])*(1-z/Lz)
T['g'] += .5*np.sin(2*np.pi*x/Lx)*(1-f['g'])

C.change_scales(1)
C['g'] = .5 +.1*np.sin(2*np.pi*x/Lx) ##.9*(1-.5*z/Lz)

In [ ]:
# Analysis
snapshots = solver.evaluator.add_file_handler('snapshots', sim_dt=0.25, max_writes=50)
snapshots.add_tasks(solver.state)
snapshots.add_task(momentum,name='momentum')
snapshots.add_task(heat,name='heat')
snapshots.add_task(salt,name='salt')
snapshots.add_task(heat_flux,name='heat_flux')
snapshots.add_task(-d3.div(d3.skew(u)), name='vorticity')

In [ ]:
# # CFL
# CFL = d3.CFL(solver, initial_dt=max_timestep, cadence=10, safety=0.5, threshold=0.05,
#              max_change=1.5, min_change=0.5, max_dt=max_timestep)
# CFL.add_velocity(u)

# # Flow properties
# flow = d3.GlobalFlowProperty(solver, cadence=10)
# flow.add_property(np.sqrt(u@u)/ν, name='Re')
# flow.add_property(f, name='phase')

In [ ]:
T.change_scales(1)
C.change_scales(1)
f.change_scales(1)
u.change_scales(1)
fig, ax = plt.subplots(2,2)
ps = {}
ps[0,0] = ax[0,0].pcolormesh(xx, zz, T['g'], cmap='RdBu_r')
ps[1,0] = ax[1,0].pcolormesh(xx, zz, C['g'], cmap='Purples')
ps[0,1] = ax[0,1].pcolormesh(xx, zz, u['g'][0], cmap='RdBu_r')
ps[1,1] = ax[1,1].pcolormesh(xx, zz, u['g'][1], cmap='RdBu_r')
for i in range(2):
    for j in range(2):
        ax[i,j].contour(xx,zz,f['g'],[.05,.5,.95],colors='k')
        plt.colorbar(ps[i,j], ax=ax[i,j])

In [ ]:
start_time = time.time()
try:
    while solver.proceed:
        if solver.iteration % 10 == 0:
            log = [f'it {solver.iteration:d}',
                   f'sim time {solver.sim_time:.2f}',
                   f'wall time {(time.time() - start_time):.1f} s',
                   f'max u {np.abs(u["g"]).max():.3f}',
                   f'max c {np.abs(C["g"]).max():.3f}',
                   f'heat {heat.evaluate()["g"][0,0]:.3f}',
                   f'salt {salt.evaluate()["g"][0,0]:.3f}',
                   ]
            logger.info(', '.join(log))
        solver.step(5e-3)
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise        